# HPDM097: Introduction to combinatorial optimisation for geospatial problems

## **Case study**: An applied facility location problem: sexual clinics in Hampshire, UK

The data you will use in the exercises in this computer lab is taken from a real facility location study conducted in Hampshire.  The data science team worked with commissioners, a public health team, and a community NHS trust in Hampshire that were reviewing the provision of sexual health clinics across the region.  The data science 'customers' had a number of concerns about the current provision of sexual health care.  One of these was the sustainability of the adult service across 28 locations.  There was a growing concern that care varied and the data scientists were asked to identify promising configurations that preserved fair access for patients.

**By the end of this lab you will have learnt how to:**

* represent a facility location problem in a general format suitable for algorithmic solution
* solve the p-median facility location problem using a brute force approach
* solve a larger p-median problem by applying an evolutionary optimisation approach
* obtain a good solution to large instances of the p-median problem by applying a Genetic Algorithm.

> Please use the conda environment `hds_logistics` when running this workbook.  You will also need to run this workbook in the same directory as `metapy`.  This is a small python package that contains the evolutionary and genetic algorithm code.

The first part of the notebook will walk you through the basics of representing a solution, generating random solutions, and evaluating a solution.  You will then be asked to complete a number of exercises using these fundamentals to locate facilities in partial and full instances of the case study.

# Imports 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#you will use itertools for enumerating all solutions in small instances.
from itertools import combinations

# `metapy` package imports

The package contains the classes and functions for the evolutionary algorithms you will use in this notebook.

In [ ]:
from metapy.evolutionary.evolutionary import (EvolutionaryAlgorithm, 
                                              MuLambdaEvolutionStrategy, 
                                              MuPlusLambdaEvolutionStrategy,
                                              GeneticAlgorithmStrategy,
                                              ElitistGeneticAlgorithmStrategy,
                                              WeightedAverageObjective,
                                              FacilityLocationPopulationGenerator,
                                              BasicFacilityLocationMutator,
                                              TournamentSelector,
                                              FacilityLocationSinglePointCrossOver)

# Import case study data

The car travel times in minutes from anonymised postcode sectors to anonymised clinic locations.

In [ ]:
travel_matrix = pd.read_csv('data/clinic_car_travel_time.csv', 
                            index_col='sector')
travel_matrix.head()

In [ ]:
#no of cases by postcode sector...

cases = pd.read_csv('data/sh_demand.csv', index_col='sector')
cases.head()

# Representing a solution

To solve the facility location problem using an an algorithm you need to decide how to represent a solution.  

You will represent a solution using a vector of length $p$ where $p <= P$.  Each element represents the index of a clinic.  For example, if we have $P = 28$ candidate locations for clinics and wish to find the best solution where $p = 4$

```python
solution = np.array([10, 0, 6, 12])
```
means that clinics which indices 10, 0, 6, and 12 are included in the solution.

To select a set of clinics from a `pandas.DataFrame` you can use the code below.  The indices are used select the column headers and the column headers are then used to select the column data from the dataframe.

In [ ]:
#the solution represents the indicies of clinics in the travel matrix
solution = np.array([10, 0, 6, 12])

#this code looks up the column names using the indicies in the solution
#if you are unsure what it does then print out `mask`
mask = travel_matrix.columns[solution]

#finally we select a restricted set of columns from the DataFrame
travel_matrix[mask]

# Constructing a random solution

It is unusual to specify an initial solution in facility location.  It is more usual to **generate** one or more random solutions as a starting place.

The function `random_solution` accepts:
* `n_candidates`: The number of candidate locations where you could place clinics (facilities)
* `p` the number of clinics to place.
* `random_seed` (optional).  Use if you wanted to recreate your results.

In [ ]:
def random_solution(n_candidates, p, random_seed=None):
    '''
    construct a random solution for the facility location
    problem.  Returns vector of length p
    '''
    #create a random number generator
    rng = np.random.default_rng(seed=random_seed)

    #sample without replacement
    solution = []
    while len(solution) < p:
        candidate = rng.integers(0, n_candidates)
        if candidate not in solution:
            solution.append(candidate)
            
    return np.array(solution)

In [ ]:
init_solution = random_solution(28, 4, random_seed=42)
init_solution

It is not strictly necessary, but if you wish to have a more efficient implementation of the above you can make use of numpy's built-in `choice` method that allows you to sample **without** replacement.

In [ ]:
def random_solution2(n_candidates, p, random_seed=None):
    '''
    construct a random solution for the facility location
    problem.  Returns vector of length p
    '''
    #create a random number generator
    rng = np.random.default_rng(seed=random_seed)
    
    #create array of candidate indexes
    candidates = np.arange(n_candidates, dtype=np.byte)
    
    #sample without replacement and return array
    return rng.choice(candidates, size=p, replace=False)

In [ ]:
init_solution = random_solution2(28, 4)
init_solution

# Evaluating a solution

There are multiple ways to formulate the objectives of a facility location problem.  With the sexual health case study you will formulate the it as a p-median problem.  Where the objective is to find the combination of facilities that minimises the weighted average car travel time to a clinic.

`metapy` contains a `WeightAverageObjective` class that accepts both the geospatial demand and travel matrix as parameters.

```python
from metapy.evolutionary.evolutionary import WeightedAverageObjective

# create an instance and pass in demand and travel times
obj = WeightedAverageObjective(cases, travel_matrix)
```

The object has a method `evaluate(solution)` that accepts a numpy vector that is your representation of the a solution.

```python
obj.evaluate(solution)
```

The code below brings these together.  Execute the cell to see the output.

In [ ]:
obj = WeightedAverageObjective(cases, travel_matrix)
obj.evaluate(solution)

# Solution of a small facility location problem by enumerating all possible combinations

For small problems it is possible to enumerate all combinations to locate the "optimal" solution.  

The function `all_combinations` below will provide a list of solutions representing an exhaustive solution space for a given problem.  For example, if the problem consisted of 10 candidate locations and you wish to evaluate solutions of size 4 then the solution space is fully represented by 210 unique solutions.

This method is straightforward for health service customers to understand and for small problems it is unusual to use a complex optimisation procedure.  As the facility location problem is NP hard, this strategy is not recommended for large scale problems.  The evaluation of the fitness of the solutions will become prohibitive as the size of the problem begins to exceed 10.

In [ ]:
def all_combinations(n_facilities, p):
    '''
    Returns all p sized combinations of an array containing
    indicies 0 to n_facilties - 1 
    '''
    facility = np.arange(n_facilities, dtype=np.uint8)
    return [np.array(a) for a in combinations(facility, p)]

In [ ]:
# size 4 combinations of 10 candidate locations = 210
comb = all_combinations(n_facilities=10, p=4)
len(comb)

In [ ]:
#take a look at index 0
comb[0]

In [ ]:
#index 209
comb[-1]

# Exercise 1: Bruteforce solution

As an example we will work with the **first six clinics** and look to find the optimal 4 sites according to the weighted average.

**Task:**
* Generate all combinations to place 4 facilities in a problem with 6 candidate locations.
* For each of the solutions calculate the weighted average travel time.
* Select the solution with the lowest weight mean car travel time.
* Illustrate all of the solutions on a bar chart.  

**Questions:**
* Does the problem have a unique "optimal" solution or are there other solutions that a health service customer might consider?
* Are there any limitations with the data that make it important to report more than a single solution?
* What happens to the travel times and solution if $p$ is reduced to 3?

In [ ]:
# your code goes here ...

# Exercise 2: Using random restarts with medium to large sized problems

When you encounter a medium to large instance of a facility location problem (or any optimisation problem) it is often tempting to move quickly onto a sophisticated solution method such as a evolutionary algorithm.  But how do you know that it is better than a simple heuristic?  

**Here you will use the random restarts algorithm (sometimes called random search) as a simple benchmark.**

Random restarts is simple heuristic.  You allocate a computational budget - either in terms of iterations or execution time - and randomly generate solutions and evaluate them. For example, you might specify an execution time limit of 10 seconds and in that time evaluate as many random solutions as possible.  

> A neat trick with this type of algorithm is to remember that it is 'Embarrassingly parallelisible'.  That is, if your machine has multiple virtual cores then you can run multiple random searches simultaneously and return the best solution.  

**Task:**
* Using the `random_solution` function defined earlier create another function `random_restarts` (or class `RandomRestarts` if you prefer) that runs for fixed number of iterations.  For example if you specify iterations as 10 then the algorithm generates 10 random solutions and evaluates them.  
* The function should return a tuple of the best cost and best solution found when minimising the weighted average car travel time.
* Problem: 20 candidate locations and a budget of 8 clinics to place.
* Try running the algorithm a few times or varying your budget. What do you notice?  What might you do to get a better understanding of the algorithms performance?

In [ ]:
#your code goes here ...

# Exercise 3: Solution using a evolutionary algorithm

## Exercise 3.1 Generating an initial population.

Basic evolutionary strategies work by mutating the most promising solutions in the population.  There are many ways to implement mutation.  Here you will use `BasicFacilityLocationMutator`.  Each element in a solution has a constant probability of mutation (by default 1 / no. of facilities in a solution, but you may wish to set this higher.).  If a facility is chosen then it is replaced by a random facility current not in the solution.

You can create a `BasicFacilityLocationMutator` as follows:

```python
mutator = FacilityLocationMutator(n_candidates=28, solution_size=4)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

To use a higher mutation rate:

```python
mutator = FacilityLocationMutator(n_candidates=28, solution_size=4
                                  mutation_rate=0.6)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

**Task**:
* Create a `FacilityLocationMutator` with 50 candidate solutions and a solution size of 10
* mutate the solution a total of 10 times and printout the generations.

In [ ]:
# your code here ...

# Exercise 3.2: Mutating a solution

Basic evolutionary strategies work by mutating the most promising solutions in the population.  There are many ways to implement mutation.  Here you will use `BasicFacilityLocationMutator`.  Each element in a solution has a constant probability of mutation (by default 1 / no. of facilities in a solution, but you may wish to set this higher.).  If a facility is chosen then it is replaced by a random facility current not in the solution.

You can create a `BasicFacilityLocationMutator` as follows:

```python
mutator = FacilityLocationMutator(n_candidates=28, solution_size=4)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

To use a higher mutation rate:

```python
mutator = FacilityLocationMutator(n_candidates=28, solution_size=4
                                  mutation_rate=0.6)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

**Task**:
* Create a `FacilityLocationMutator` with 50 candidate solutions and a solution size of 10
* mutate the solution a total of 10 times and printout the generations.


In [ ]:
#your code goes here ...

# Exercise 3.3: The $(\mu, \lambda)$ and $(\mu+\lambda)$ evolutionary strategies

A random initial population and a mutation operator provide the ingredients for the two basic evolutionary strategies $(\mu, \lambda)$ and $(\mu+\lambda)$.

You will use the `metapy` package.  In particular:


**Task:**
* Run two evolutionary algorithms with $(\mu, \lambda)$ and $(\mu+\lambda)$  strategies respectively.  The code is  provided below.
* Investigate the parameters requried when creating `MuPlusLambdaEvolutionStrategy` and `MuLambdaEvolutionStrategy`
* Use a problem size of 28 candidate locations and 14 facilities
* Initially try $mu = 10$ and $\lambda = 200$. 
* Using a random initial population evolve for 50 generations.

**Hints**:
* Evolutionary strategies are computationally expensive.  Expect a 50 generation algorithm to take 20-45 seconds on your machine.

**Questions:**
* How do your new solutions and algorithms compare to `random_restarts?`
* How do the algorithms appear to compare to each other?
* What must your remember about evolutionary algorithms when evaluating their performance?

In [ ]:
%%time

#example solution

#Evolutionary Algorithm - (mu+lambda) strategy
n_candidates = 28
n_facilities = 14

mu = 10
_lambda = 200

#objective
objective = WeightedAverageObjective(cases, travel_matrix)

#initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

#mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

#evolutionary stategy
strategy = MuPlusLambdaEvolutionStrategy(mu, _lambda, mutator)

#solver object
solver = EvolutionaryAlgorithm(init, objective, _lambda, strategy, 
                               maximisation=False, generations=50)

print("\nRunning (mu + lambda) evolutionary alg...")
solver.solve()

print("\n** (MU+LAMBDA) OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(sorted(solver.best_solution))

In [ ]:
%%time

#Evolutionary Algorithm - (mu,lambda) strategy
n_candidates = 28
n_facilities = 14

mu = 10
_lambda = 200

#objective
objective = WeightedAverageObjective(cases, travel_matrix)

#initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

#mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

#evolutionary stategy
strategy = MuLambdaEvolutionStrategy(mu, _lambda, mutator)

#solver object
solver = EvolutionaryAlgorithm(init, objective, _lambda, strategy, 
                               maximisation=False, generations=50)

print("\nRunning (mu, lambda) evolutionary alg...")
solver.solve()

print("\n** (MU,LAMBDA) OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(sorted(solver.best_solution))

# Exercise 4: Locating facilities using a full Genetic Algorithm (GA)

Now that you have warmed up using  $(\mu, \lambda)$ and $(\mu+\lambda)$  it is time to move onto a full GA.  This means you need to take account of two further steps.

* A selection operator for breeding - in this instance you will use the provided `TournamentSelector`
* A crossover operator for breeding - you will use `FacilityLocationSinglePointCrossover`

See lecture slides for details of how these work.

`metapy` provides standard and elitist GA strategies.  The code provided in this task demonstrates how these are instantiated and used to solve the sexual health clinic facility location problem.

**Task:**

The two code blocks below have been provided to demonstrate how to use run the `metapy` implementations of a GA.  Note that these are similar to the two basic evolutionary strategies you used in the previous exercise.  

* Run `GeneticAlgorithmStrategy` and `ElitistGeneticAlgorithmStrategy` using the parameters provided


**Questions**:

* Are you satisfied with the results?  If not you could try changing/tuning the parameters:
    * lambda and mu
    * the number of generations (note this will start to get slow with large numbers)
    * the mutation rate
    * (You could also work with a smaller problem size to speed things up).
    
* Which of the algorithms used fin this case study do you prefer the most and why?

* Ultimately, in practice, if you are not satisfied with the performance of the GA you might need to code new cross-over and mutation operators! 

    

In [ ]:
%%time

#Evolutionary Algorithm - Genetic Algorithm strategy

n_candidates = 28
n_facilities = 14

_lambda = 200
#objective
objective = WeightedAverageObjective(cases, travel_matrix)

#initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

#mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

#cross over operator
x_over = FacilityLocationSinglePointCrossOver()

#GA strategy
strategy = GeneticAlgorithmStrategy(_lambda, 
                                    selector=TournamentSelector(),
                                    xoperator=x_over,
                                    mutator=mutator)


solver = EvolutionaryAlgorithm(init, objective,_lambda, strategy, 
                               maximisation=False, generations=50)
print("\nRunning Genetic Algorithm")
solver.solve()

print("\n** GA OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(solver.best_solution)

In [ ]:
%%time 

#Evolutionary Algorithm - Elistist Genetic Algorithm strategy

n_candidates = 28
n_facilities = 14

#GA parameters
mu = 10
_lambda = 200

#objective
objective = WeightedAverageObjective(cases, travel_matrix)

#initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

#mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

#cross over operator
x_over = FacilityLocationSinglePointCrossOver()

#GA strategy
strategy = ElitistGeneticAlgorithmStrategy(mu,
                                           _lambda, 
                                           selector=TournamentSelector(),
                                           xoperator=x_over,
                                           mutator=mutator)


solver = EvolutionaryAlgorithm(init, objective,_lambda, strategy, 
                               maximisation=False, generations=50)
print("\nRunning Elitist Genetic Algorithm")
solver.solve()

print("\n** ELITIST GA OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(solver.best_solution)

# End